In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm 

#load course IDs
course_ids = pd.read_csv('data/course_IDs.csv')
course_ids = course_ids['course_ID'].tolist()
course_ids = [str(i) for i in course_ids]


In [2]:
TESTING = False

import random
random.seed(123)
course_ids_test = random.sample(course_ids, 20)


In [3]:
def get_details(soup, url):
    # get the course name
    title_article = soup.find('article', class_='content')
    year = soup.find('div', class_='edition').text.strip()
    
    course_title = soup.find('h1').text.strip()

    try:
        language = soup.find('dt', string='Language')
        language = language.find_next_sibling().text.strip()
    except:
        language = 'None'
    
    try:
        college = soup.find('dt', string='Faculty')
        college = college.find_next_sibling().text.strip()
    except:
        college = 'None'

    try:
        lecturers = soup.find('dt', string={'Teacher', 'Teachers'})
        lecturers = lecturers.find_next_sibling().text.strip()
        lecturers = lecturers.split('\n')
    except:
        lecturers = 'None'

    try:
        lecturers_urls = soup.find('dt', string={'Teacher', 'Teachers'})
        lecturers_urls_container = lecturers_urls.find_next_sibling()
        lecturers_urls = lecturers_urls_container.find_all('a')
        #get links
        if len(lecturers_urls) == 0:
            lecturers_urls = 'None'
        else:
            lecturers_urls = [i['href'] for i in lecturers_urls]
    except:
        lecturers_urls = 'None'

    try:
        programmes_prev = soup.find('h2', string='Part of')
        programmes_container = programmes_prev.find_next_sibling('ul')
        programmes = programmes_container.find_all('a')
        #get links
        programmes = [i['href'] for i in programmes]
    except:
        programmes = 'None'

    try:
        text_container = soup.find('div', class_='wrapper main clearfix')
        text = text_container.find('article', class_='content').text.strip()
    except:
        text = 'None'

    #create dictionary from course details
    course_details = {
        'url': url,
        'year': year,
        'course_title': course_title,
        'language': language,
        'college': college,
        'lecturers_urls': lecturers_urls,
        'lecturers': lecturers,
        'programmes': programmes,
        'text': text
    }

    return course_details

In [5]:
url = 'https://studiegids.universiteitleiden.nl/en/courses/'

# create empty df
course_details_df = pd.DataFrame(columns=['url', 'year','course_title','language','college','lecturers_urls','lecturers','programmes','text'])

# define the number of chunks
num_chunks = 10

# calculate the chunk size
chunk_size = len(course_ids) // num_chunks

# loop over the chunks
for chunk_index in range(num_chunks):
    # get the start and end indices for the current chunk
    start_index = chunk_index * chunk_size
    end_index = (chunk_index + 1) * chunk_size
    
    # get the course IDs for the current chunk
    chunk_course_ids = course_ids[start_index:end_index]
    
    
    #print diagnosic info
    print(f'Processing chunk {chunk_index + 1} of {num_chunks}...')
    print(f'Start index: {start_index}')    
    print(f'End index: {end_index}')
    print(f'Number of course IDs in chunk: {len(chunk_course_ids)}')
    print()

        # loop over the course IDs in the current chunk
    for i in tqdm(chunk_course_ids if TESTING else chunk_course_ids):
        # get the html code of the course page
        url_new = url + i
        response = requests.get(url_new)
        soup = BeautifulSoup(response.text, 'html.parser')

        course_details = get_details(soup, url_new)

        # append dictionary to df
        course_details_df = course_details_df._append(course_details, ignore_index=True)

    # save df to csv
    csv_filename = f'data/course_details_part{chunk_index + 1}.csv'
    course_details_df.to_csv(csv_filename, index=False)
    course_details_df = pd.DataFrame(columns=['url', 'year','course_title','language','college','lecturers_urls','lecturers','programmes','text'])


Processing chunk 1 of 10...
Start index: 0
End index: 5787
Number of course IDs in chunk: 5787



  0%|          | 0/5787 [00:00<?, ?it/s]

ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))